In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_landsat()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_h,wp_le,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,2.605590,13.643902,0.377924,0.413754,0.594708,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,3.459300,13.498761,0.381184,0.420529,0.597960,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,2.117474,14.987823,0.384444,0.427304,0.601212,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,4.636145,11.355851,0.387704,0.434079,0.604465,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,3.448946,10.943102,0.390963,0.440854,0.607717,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 24 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 non-null float64
wp_LST.night    1028 non-null float64
LW_IN.wp        1028 non-null float64
LW_IN.si        1028 non-null float64
LW_IN           1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_le"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.97946605826
CV Scores:  0.984290788732, 0.972352059238, 0.974218967012, 0.97568905023, 0.984173202327, 0.978217729543, 0.983256614254, 0.982206699828, 0.977154734825, 0.983100736616
OOB score: 0.980418499946
Feature Importances:
('wp_LST.night', 0.23360621054210778)
('air_temp', 0.1920530565074435)
('wp_evi', 0.14983717819938167)
('PET', 0.10461297117671622)
('wp_lswi2', 0.079812698083479447)
('wp_ndvi', 0.076776456646895086)
('wp_LST.day', 0.071397194583898815)
('VPD', 0.043507363245334746)
('sw_in', 0.032260360625709888)
('LW_IN.wp', 0.0079115692534603901)
('wind_speed', 0.0074049663075853624)
('precip', 0.00081997482798727608)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.981089410501
CV Scores:  0.985706138821, 0.970321312662, 0.980363979976, 0.976528412717, 0.985882215147, 0.978984476805, 0.984997651982, 0.984398490642, 0.982716435664, 0.980994990592
Feature Importances:
('wp_evi', 0.16300704025189597)
('wp_lswi2', 0.14468108150060582)
('wp_LST.night', 0.10404209112909263)
('air_temp', 0.098715590018268171)
('PET', 0.091808866918092619)
('wp_LST.day', 0.076499092565469023)
('wp_ndvi', 0.072004191025660469)
('sw_in', 0.065278069356165855)
('LW_IN.wp', 0.062868869997299362)
('wind_speed', 0.055873350511164599)
('VPD', 0.051275361611701306)
('precip', 0.013946395114584269)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.964916263172
CV Scores:  0.972985776862, 0.949672954982, 0.960338118866, 0.958991559191, 0.958721865478, 0.971658546435, 0.962153660272, 0.969525859105, 0.969559140235, 0.97555515029


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 64.04565
Step #501, epoch #50, avg. loss: 1.76374
Step #1001, epoch #100, avg. loss: 0.65779
Step #1501, epoch #150, avg. loss: 0.42201
Step #2001, epoch #200, avg. loss: 0.30190
Step #2501, epoch #250, avg. loss: 0.22791
Step #3001, epoch #300, avg. loss: 0.19731
Step #3501, epoch #350, avg. loss: 0.16175
Step #4001, epoch #400, avg. loss: 0.15168
Step #4501, epoch #450, avg. loss: 0.12926
Step #1, avg. loss: 48.25198
Step #501, epoch #50, avg. loss: 1.74794
Step #1001, epoch #100, avg. loss: 0.66091
Step #1501, epoch #150, avg. loss: 0.40479
Step #2001, epoch #200, avg. loss: 0.28653
Step #2501, epoch #250, avg. loss: 0.22869
Step #3001, epoch #300, avg. loss: 0.20415
Step #3501, epoch #350, avg. loss: 0.17588
Step #4001, epoch #400, avg. loss: 0.16091
Step #4501, epoch #450, avg. loss: 0.14160
Step #1, avg. loss: 55.22949
Step #501, epoch #50, avg. loss: 1.80768
Step #1001, epoch #100, avg. loss: 0.71968
Step #1501, epoc

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x117159488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)